#  广播服务器

In [ ]:
from typing import List
from twisted.internet.protocol import Protocol, Factory
from twisted.internet.endpoints import TCP4ServerEndpoint
from twisted.internet import reactor

clients: List[Protocol] = []


class ServerFactory(Factory):
    def __init__(self):
        self.numProtocols = 0

    def buildProtocol(self, addr):
        return Server(self)


class Server(Protocol):
    def __init__(self, factory: ServerFactory):
        self.factory = factory

    def connectionMade(self):
        self.factory.numProtocols += 1
        self.transport.write('您是第 {} 个客户端'.format(self.factory.numProtocols).encode('utf-8'))
        print('new connect: {} - {}'.format(self.factory.numProtocols, self.transport.getPeer()))
        clients.append(self)
        print(clients)

    def connectionLost(self, reason):
        clients.remove(self)
        print("Lost connect {}".format(self))

    def dataReceived(self, data):
        if data == 'close':
            self.transport.loseConnection()
            print('server closed {}'.format(self))
        else:
            print('[ID: {}]>>> {}'.format(self, data))
            for client in clients:
                if client != self:
                    client.transport.write(data)


if __name__ == '__main__':
    endpoint = TCP4ServerEndpoint(reactor, 8007)
    endpoint.listen(ServerFactory())
    print("....等待链接..")
    reactor.run()

# Twisted 客户端

In [ ]:
import datetime
from twisted.internet.protocol import Protocol, ClientFactory
from twisted.internet import reactor


class Echo(Protocol):
    def connectionMade(self):
        print('Connected to the server!')

    def dataReceived(self, data: bytes):
        print('got message: ', data.decode('utf-8'))
        reactor.callLater(2, self.say_hello)

    def connectionLost(self, reason):
        print("Disconnected from the server")

    def say_hello(self):
        if self.transport.connected:
            self.transport.write("hello, {}".format(datetime.datetime.now()).encode('utf-8'))


class EchoClientFactory(ClientFactory):
    def __init__(self):
        self.protocol = None

    def startedConnecting(self, connector):
        print('Started to connect.')

    def buildProtocol(self, addr):
        self.protocol = Echo()
        return self.protocol

    def clientConnectionLost(self, connector, reason):
        print('Lost connection, Reason:', reason)

    def clientConnectionFailed(self, connector, reason):
        print('Connection fail, Reason:', reason)


if __name__ == '__main__':
    host = '127.0.0.1'
    port = 8007
    factory = EchoClientFactory()
    reactor.connectTCP(host, port, factory)
    reactor.run()